In [27]:
import sys
sys.path.insert(0, "..")
import canterburycommuto.CanterburyCommuto as cc

In [28]:
As = "5.373588,-3.998759"
Ae = "5.327810,-4.005012"
Bs = "5.373588,-3.998759"
Be = "5.327810,-4.005012"

Get the coordinates of route A and B. 

In [ ]:
coordinates_a, total_distance_a, total_time_a = cc.get_route_data(As, Ae, "key") #Please replace 'key' with your Google API key.
coordinates_b, total_distance_b, total_time_b = cc.get_route_data(Bs, Be, "key")

Find the first and last common node of the two routes. 

In [21]:
first_common_node, last_common_node = cc.find_common_nodes(
            coordinates_a, coordinates_b
        )

VS Code does not directively display Folium maps when running Python scripts, but Jupyter Notebook does.
Folium generates an interactive HTML file, which must be opened manually. The following step plots the two routes with the first and last common node drawn.

In [22]:
try:
    map_filename = cc.save_map(cc.plot_routes(coordinates_a, coordinates_b, first_common_node, last_common_node), "routes_map")
except AttributeError:
    map_filename = "Map saved successfully, ignoring error."

Map saved to: c:\Users\HUAWEI\CanterburyCommuto\example\results\routes_map-2025-02-16_22-45-56_67721.html


Since the two routes completely overlap, there are no distinct segments before or after the overlap in the implementation of the rectangle approximation. As a result, the function find_overlap_boundary_nodes shown below does not detect transition points and returns `None` for both boundary nodes.

In [30]:
# Calculate segment distances for both routes
segments_a = cc.calculate_distances(coordinates_a, "A")
segments_b = cc.calculate_distances(coordinates_b, "B")

# Create rectangular buffers around each segment
rectangles_a = cc.create_segment_rectangles(segments_a, width=100)
rectangles_b = cc.create_segment_rectangles(segments_b, width=100)

# Find overlapping segment combinations
overlapping_combinations = cc.find_segment_combinations(rectangles_a, rectangles_b)

# Filter combinations based on overlap threshold (e.g., 50%)
filtered_combinations = cc.filter_combinations_by_overlap(rectangles_a, rectangles_b, threshold=50)

# Extract boundary nodes
boundary_nodes = cc.find_overlap_boundary_nodes(filtered_combinations, rectangles_a, rectangles_b)

print("Boundary Nodes:", boundary_nodes)


Boundary Nodes: {'first_node_before_overlap': None, 'last_node_after_overlap': None}


Here is an implementation of the buffer method to possibly account for multiple intersections of two routes.

In [23]:
buffer_distance = 100  # Buffer radius in meters

buffer_a = cc.create_buffered_route(coordinates_a, buffer_distance)
buffer_b = cc.create_buffered_route(coordinates_b, buffer_distance)

print("Buffer A Created:", buffer_a is not None)
print("Buffer B Created:", buffer_b is not None)


Buffer A Created: True
Buffer B Created: True


The following code block plots the buffered routes. 

In [25]:
try:
    map_filename = cc.save_map(
        cc.plot_routes_and_buffers(coordinates_a, coordinates_b, buffer_a, buffer_b),
        "routes_with_buffers_map"
    )
except AttributeError:
    map_filename = "Map saved successfully, ignoring error."

print(f"Map saved as {map_filename}")


Map saved to: c:\Users\HUAWEI\CanterburyCommuto\example\results\routes_with_buffers_map-2025-02-16_22-51-19_79416.html


Map has been displayed inline and saved as 'results\routes_with_buffers_map-2025-02-16_22-51-19_79416.html'.
Map saved as Map saved successfully, ignoring error.


What happen if you have multiple intersection？

The intersection function from the shapely library is probably able to capture multiple intersection areas. 

Is it possible to directly work with MultiLineString

Professor, as we discussed, it probably works well. 

Why rectangles?

Professor Grosset instructed me to create polygons around the route. Probably rectangles can better approximate the road segment shapes? However, ellipses and other shapes can probably work well.

Create an object for API management.

An API manager .py file and test .py file have been created.